In [1]:
import cv2
import os
import re
import warnings
import keras
from keras.applications.imagenet_utils import preprocess_input
from keras.backend.tensorflow_backend import set_session
from keras.models import Model
from keras.preprocessing import image
from keras.backend import tensorflow_backend
import matplotlib.pyplot as plt
import numpy as np
from scipy.misc import imread
import tensorflow as tf

from ssd import SSD300
from ssd_utils import BBoxUtility

%matplotlib inline
plt.rcParams['figure.figsize'] = (8, 8)
plt.rcParams['image.interpolation'] = 'nearest'

np.set_printoptions(suppress=True)

#config = tf.ConfigProto()
#config.gpu_options.per_process_gpu_memory_fraction = 0.45
#set_session(tf.Session(config=config))

/home/tsunei/anaconda3/envs/tf/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
config = tf.ConfigProto(gpu_options = tf.GPUOptions(allow_growth = True))
session = tf.Session(config = config)
tensorflow_backend.set_session(session)

In [3]:
voc_classes = ['jinkou','sizen']
NUM_CLASSES = len(voc_classes) + 1

In [4]:
input_shape=(300, 300, 3)
warnings.filterwarnings('ignore')
model = SSD300(input_shape, num_classes=NUM_CLASSES)
model.load_weights('./checkpoints/weights.100-1.97.hdf5', by_name=True)
#model.load_weights('./weights_SSD300.hdf5', by_name=True)
bbox_util = BBoxUtility(NUM_CLASSES)

In [5]:
inputs = []
images = []
inputs2 = []
images2 = []

def list_pictures(directory, ext='jpg|jpeg|bmp|png|ppm'):
    return [os.path.join(root, f)
            for root, _, files in os.walk(directory) for f in files
            if re.match(r'([\w]+\.(?:' + ext + '))', f.lower())]

print('テストデータを読み込み中…')
for filepath in list_pictures('../master/data/CNN_data/test/jinkou/'):   
#for filepath in list_pictures('../master/data/CNN_data/test/sizen/'):   
    warnings.filterwarnings('ignore')
    img = image.load_img(filepath, target_size=(300, 300))
    img = image.img_to_array(img)
    images.append(imread(filepath))
    inputs.append(img.copy())
    
inputs = preprocess_input(np.array(inputs))
     
for filepath in list_pictures('../master/data/CNN_data/test/sizen/'):   
    warnings.filterwarnings('ignore')
    img2 = image.load_img(filepath, target_size=(300, 300))
    img2 = image.img_to_array(img2)
    images2.append(imread(filepath))
    inputs2.append(img2.copy())

inputs2 = preprocess_input(np.array(inputs2))

print('テストデータを読み込み終了')

テストデータを読み込み中…
テストデータを読み込み終了


In [6]:
preds = model.predict(inputs, batch_size=1, verbose=1)
preds2 = model.predict(inputs2, batch_size=1, verbose=1)
#preds = model.predict(inputs, batch_size=1)
#print(preds.shape)
#print(preds[0])

1200/1200 [==============================] - 16s 14ms/step


In [7]:
results = bbox_util.detection_out(preds)
results2 = bbox_util.detection_out(preds2)
#print(results[3])

In [ ]:
def detector(images_eva, result, img):
    no_feature = 0
    count_jinkou = 0
    count_sizen = 0
    for i, img in enumerate(images_eva):
        # Parse the outputs.
        if len(results[i])==0:
            no_feature += 1
            continue


        det_label = result[i][:, 0]
        det_conf = result[i][:, 1]
        det_xmin = result[i][:, 2]
        det_ymin = result[i][:, 3]
        det_xmax = result[i][:, 4]
        det_ymax = result[i][:, 5]           


        # Get detections with confidence higher than 0.6.
        top_indices = [i for i, conf in enumerate(det_conf) if conf >= 0.6]

        top_conf = det_conf[top_indices]
        top_label_indices = det_label[top_indices].tolist()
        top_xmin = det_xmin[top_indices]
        top_ymin = det_ymin[top_indices]
        top_xmax = det_xmax[top_indices]
        top_ymax = det_ymax[top_indices]

        colors = plt.cm.hsv(np.linspace(0, 1, 21)).tolist()

        plt.imshow(img / 255.)
        currentAxis = plt.gca()

        sum_jinkou = 0
        sum_sizen = 0

        for i in range(top_conf.shape[0]):
            xmin = int(round(top_xmin[i] * img.shape[1]))
            ymin = int(round(top_ymin[i] * img.shape[0]))
            xmax = int(round(top_xmax[i] * img.shape[1]))
            ymax = int(round(top_ymax[i] * img.shape[0]))
            score = top_conf[i]
            label = int(top_label_indices[i])
            label_name = voc_classes[label - 1]
            if label_name == 'jinkou':
                sum_jinkou += ((xmax - xmin) * (ymax - ymin)) 
            else:
                sum_sizen += ((xmax - xmin) * (ymax - ymin)) 

            display_txt = '{:0.2f}, {}'.format(score, label_name)
            coords = (xmin, ymin), xmax-xmin+1, ymax-ymin+1
            color = colors[label]
            #currentAxis.add_patch(plt.Rectangle(*coords, fill=False, edgecolor=color, linewidth=2))
            #currentAxis.text(xmin, ymin, display_txt, bbox={'facecolor':color, 'alpha':0.5})


        if sum_jinkou > sum_sizen:
            count_jinkou += 1
            #print(sum_jinkou, ':', sum_sizen, '-> jinkou', count_jinkou)
        elif sum_jinkou < sum_sizen:
            count_sizen += 1
            #print(sum_jinkou, ':', sum_sizen, '-> sizen', count_sizen)
            


        #plt.show()
    print('自然物画像：', count_sizen / 1200 * 100, '%')
    print('人工物画像：', count_jinkou / 1200 * 100, '%')

        
print('人工物画像の正解率')
detector(images, results, img)
#print('自然物画像の正解率')
#detector(images2, results2, img2)

人工物画像の正解率
自然物画像： 12.333333333333334 %
人工物画像： 87.58333333333333 %
